In [1]:
import pandas as pd
import re

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.set_option('display.max_colwidth',None)

import requests
from bs4 import BeautifulSoup

import json

from tqdm import tqdm
import re
from tabulate import tabulate

In [2]:
def extract_data():
    
    movie_name=[]
    movie_year=[]
    movie_rating=[]
    movie_cast=[]
    movie_director=[]
    movie_users=[]
    
    movie_genre=[]
    movie_production=[]
    movie_release_month=[]
    
    movie_budget=[]
    movie_collection_US_Canada=[]
    movie_weekend_collection= []
    movie_collection_worldwide=[]
    
    
    movie_duration=[]
    
    movie_meta_score=[]
    
    movie_origin=[]

    def total_users(users):
        users= users.replace(',','')
        pattern= re.compile(r' \d+')
        matches= pattern.finditer(users)

        for match in matches:
            return(match.group().strip())
        
        
    
    def only_currency(str):
        return round((float((re.findall(r'[\d,]+', str))[0].replace(',',''))),2)
    
    
    url= 'https://www.imdb.com/chart/top/'
    page = requests.get(url)

    soup = BeautifulSoup(page.text, 'html.parser')

    movies = soup.find('tbody', class_= 'lister-list' )
    all_tr_movies = movies.find_all('tr')

    
    for x in tqdm(all_tr_movies):
        y = x.find_all('td')
        y=y[1:3]
        name = y[0].find('a').text
        cast= y[0].find('a')['title']
        year = y[0].find('span').text
        rating = y[1].find('strong').text
        users= y[1].find('strong')['title']
        
        
    
        movie_name.append(name)
        
        movie_cast.append(cast.split(',',1)[1].strip())
        movie_director.append(cast.split(',',1)[0].replace('(dir.)','').strip())
        
        movie_year.append(int((year).replace('(','').replace(')','')))
        movie_users.append(int(total_users(users)))
        movie_rating.append(float(rating))
        
        
        
        movie_info = x.find('td', class_ = 'titleColumn').find('a')['href']
        movie_info = 'https://www.imdb.com/'+ str(movie_info)
    
        page = requests.get(movie_info)
        soup = BeautifulSoup(page.text, 'html.parser')
        
        soup_2= soup.find('script', type= 'application/json')
        json_data = json.loads(soup_2.contents[0])
        
        movie_release_month.append((json_data['props']['pageProps']['aboveTheFoldData']['releaseDate']['month']))
        
        movie_genre.append((json_data['props']['pageProps']['aboveTheFoldData']['genres']['genres'][0]['text']))
        
        movie_production.append((json_data['props']['pageProps']['aboveTheFoldData']['production']['edges'][0]['node']['company']['companyText']['text']))

        soup_3 = soup.find_all('div', class_ = 'sc-f65f65be-0 ktSkVi')
       
        for p in soup_3:
            if 'title-boxoffice-section' in str(p):
                soup_3= p  
            
                
        try:
            soup_4 = soup_3.find_all('span', class_ = 'ipc-metadata-list-item__label') #detail 
            
        
            a=[]
            for q in soup_4:
                a.append(q.text)
        
        
            soup_3= soup_3.find_all('span', class_ = 'ipc-metadata-list-item__list-content-item')
       
            if 'Budget' in a:
                   movie_budget.append(only_currency(soup_3[0].text))
                   movie_origin.append(soup_3[0].text)
                   
            else:
                movie_budget.append(None)
                soup_3.insert(0,None)
                movie_origin.append(None)
                
            
            if 'Gross US & Canada' in a:
                movie_collection_US_Canada.append(only_currency(soup_3[1].text))
            else:
                movie_collection_US_Canada.append(None)
                soup_3.insert(1,None)
            
            if 'Opening weekend US & Canada' in a:
                movie_weekend_collection.append(only_currency(soup_3[2].text))
            else:
                movie_weekend_collection.append(None)
                soup_3.insert(2,None)
                soup_3.insert(3,None)
            
            if 'Gross worldwide' in a:
                movie_collection_worldwide.append(only_currency(soup_3[4].text))
            else:
                movie_collection_worldwide.append(None)
                soup_3.insert(4,None)
            
            
        except Exception as e:
            movie_budget.append(None)
            movie_collection_US_Canada.append(None)
            movie_weekend_collection.append(None)
            movie_collection_worldwide.append(None)
            movie_origin.append(None)


            print(e)
            
            
        soup_5 = soup.find_all('ul', class_ = 'ipc-inline-list ipc-inline-list--show-dividers sc-8c396aa2-0 kqWovI baseAlt')
       
        for g in soup_5:
            if 'hero-title-block__metadata' in str(g):
                soup_5= g
        soup_5= (soup_5).find_all('li', class_= 'ipc-inline-list__item')  
        
        s=(soup_5[-1]).text
        time = re.findall('\d+', s)
        if len(time)==1:
            duration = int(time[0])*60
        else:
            duration= int(time[0])*60 + int(time[1])
        
        movie_duration.append(duration)
        
        meta_score_tag = soup.find('span' , class_ = 'score-meta')
        if meta_score_tag !=None:
            meta_score = meta_score_tag.text
            movie_meta_score.append(int(meta_score))
        else:
            movie_meta_score.append(None)
        
        
        
        
        
    return(pd.DataFrame(list(zip(movie_name, movie_year, movie_release_month, movie_duration, movie_cast, movie_director, movie_users, 
                        movie_production, movie_genre, movie_budget,  movie_collection_US_Canada, movie_weekend_collection,
                        movie_collection_worldwide, movie_meta_score,movie_origin,movie_rating )),
               columns =['Name', 'Year', 'Month', 'duration','Cast','Director', 'Users', 'Production', 'Genre' ,'Budget','US_Canada_Collection', 
                         'Weekend_Collection', 'Worlwide_Collection', 'Meta_Score','Origin_Currency_Symbol','Rating']) )
      
    
    

# soup is page we get after we click on any movie present on the page - top 250

In [3]:
choice = input('Enter "YES" to extract the imdb data for top 250 movies')
if choice == 'YES':
    df = extract_data()
    print(tabulate(df, headers = 'keys', tablefmt = 'psql'))

Enter "YES" to extract the imdb data for top 250 moviesYES


 43%|██████████████████████████████████▌                                             | 108/250 [02:13<03:21,  1.42s/it]

ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?


 95%|████████████████████████████████████████████████████████████████████████████▏   | 238/250 [04:40<00:15,  1.32s/it]

ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [04:53<00:00,  1.17s/it]

+-----+----------------------------------------------------------------------+--------+---------+------------+--------------------------------------------+----------------------------------+---------+----------------------------------------------------------------+-----------+------------------+------------------------+----------------------+-----------------------+--------------+-----------------------------+----------+
|     | Name                                                                 |   Year |   Month |   duration | Cast                                       | Director                         |   Users | Production                                                     | Genre     |           Budget |   US_Canada_Collection |   Weekend_Collection |   Worlwide_Collection |   Meta_Score | Origin_Currency_Symbol      |   Rating |
|-----+----------------------------------------------------------------------+--------+---------+------------+----------------------------------------

In [47]:
#df.to_excel (r'C:\Users\Dell\Desktop\250_IMDB_final.xlsx', index = False, header=True)

In [4]:
display(df)

,Name,Year,Month,duration,Cast,Director,Users,Production,Genre,Budget,US_Canada_Collection,Weekend_Collection,Worlwide_Collection,Meta_Score,Origin_Currency_Symbol,Rating
0,The Shawshank Redemption,1994,10,142,"Tim Robbins, Morgan Freeman",Frank Darabont,2647137,Castle Rock Entertainment,Drama,2.500000e+07,28767189.0,727327.0,2.888450e+07,81.0,"$25,000,000 (estimated)",9.2
1,The Godfather,1972,3,175,"Marlon Brando, Al Pacino",Francis Ford Coppola,1834929,Paramount Pictures,Crime,6.000000e+06,136381073.0,302393.0,2.503418e+08,100.0,"$6,000,000 (estimated)",9.2
2,The Dark Knight,2008,7,152,"Christian Bale, Heath Ledger",Christopher Nolan,2619082,Warner Bros.,Action,1.850000e+08,534987076.0,158411483.0,1.006234e+09,84.0,"$185,000,000 (estimated)",9.0
3,The Godfather Part II,1974,12,202,"Al Pacino, Robert De Niro",Francis Ford Coppola,1257560,Paramount Pictures,Crime,1.300000e+07,47834595.0,171417.0,4.796192e+07,90.0,"$13,000,000 (estimated)",9.0
4,12 Angry Men,1957,4,96,"Henry Fonda, Lee J. Cobb",Sidney Lumet,781564,Orion-Nova Productions,Crime,3.500000e+05,NaN,NaN,9.550000e+02,96.0,"$350,000 (estimated)",8.9
5,Schindler's List,1993,2,195,"Liam Neeson, Ralph Fiennes",Steven Spielberg,1341680,Universal Pictures,Biography,2.200000e+07,96898818.0,656636.0,3.221612e+08,94.0,"$22,000,000 (estimated)",8.9
6,The Lord of the Rings: The Return of the King,2003,12,201,"Elijah Wood, Viggo Mortensen",Peter Jackson,1823038,New Line Cinema,Action,9.400000e+07,378251207.0,72629713.0,1.146436e+09,94.0,"$94,000,000 (estimated)",8.9
7,Pulp Fiction,1994,10,154,"John Travolta, Uma Thurman",Quentin Tarantino,2025861,Miramax,Crime,8.000000e+06,107928762.0,9311882.0,2.139288e+08,94.0,"$8,000,000 (estimated)",8.8
8,The Lord of the Rings: The Fellowship of the Ring,2001,12,178,"Elijah Wood, Ian McKellen",Peter Jackson,1849270,New Line Cinema,Action,9.300000e+07,316115420.0,47211490.0,8.981470e+08,92.0,"$93,000,000 (estimated)",8.8
9,"The Good, the Bad and the Ugly",1966,12,178,"Clint Eastwood, Eli Wallach",Sergio Leone,755509,Produzioni Europee Associate (PEA),Adventure,1.200000e+06,25100000.0,NaN,2.525389e+07,90.0,"$1,200,000 (estimated)",8.8
